In [1]:
import os
import discord
from discord.ext import commands
from discord import Intents, Client, Message
from Bot_Responses import get_response
from random import choice, randint
import pandas as pd

In [2]:
# Your bot's token obtained from the Discord Developer Portal
TOKEN = 'MTIzNzU3NTA3NjE3ODY5NDE2NA.G9TNWi.f3AKeys5HytGHovD48OsA8ty-k2g-YV-wr6l2k'
CHANNEL_ID = 1237890312937148527

# Initialize the Discord client and bot
intents: intents = Intents.default()
intents.message_content = True
client = Client(intents = intents)
client.token = TOKEN

bot = commands.Bot(command_prefix = "", intents = discord.Intents.all())

i=0
j=0
k=0

In [3]:
# Message Function
@client.event
async def send_message(message: Message, user_message:str):
    if not user_message:
        print('Message was empty because intents were not enabled...prob')
        return
    if is_private := user_message[0] == '?':
        user_message = user_message[1:]

    try:
        response: str = get_response(user_message)
        await message.author.send(response) if is_private else await message.channel.send(response)
    except Exception as e:
        print(e)
    

In [4]:
#Handle start up for bot
@client.event
async def on_ready():
    print(f'{client.user} has connected to Discord!')
    channel = bot.get_channel(CHANNEL_ID)


In [5]:
#Handle Messages
@client.event
async def on_message(message: Message):
    #bot talking to itself case and checking for previous bot statement for back and forth
    if message.author == client.user:
        return


    username: str=str(message.author)
    user_message:str = message.content
    channel: str = str(message.channel)

    print(f'[{channel}], {username}: "{user_message}"')
    await send_message(message, user_message)

        

In [6]:
@client.event
async def shutdown(context):
    exit()

In [7]:
# Run the bot
def main() -> None:
    client.run(token = TOKEN)

In [8]:
import asyncio
asyncio.get_event_loop().create_task(client.start(TOKEN))

<Task pending name='Task-5' coro=<Client.start() running at C:\Users\olivi\AppData\Local\Programs\Python\Python312\Lib\site-packages\discord\client.py:756>>

Property Bot#2662 has connected to Discord!
[bot-channel], .oliviayang: "hello"
[bot-channel], .oliviayang: "help"
